<a href="https://colab.research.google.com/github/prajwalraju/GoogleColab/blob/main/Wav2Lip_TenDeepfake_eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title <h1>Install All dependency</h1>
#@markdown * Install dependencies
#@markdown * Download models
#@markdown * Mount google drive
!rm -rf /content/sample_data
!mkdir /content/sample_data

!git clone https://github.com/zabique/Wav2Lip

#download the pretrained model
!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O '/content/Wav2Lip/checkpoints/wav2lip_gan.pth'
!wget 'https://iiitaphyd-my.sharepoint.com/:u:/g/personal/radrabha_m_research_iiit_ac_in/Eb3LEzbfuKlJiR600lQWRxgBIY27JZg80f7V9jtMfbNDaQ?e=TBFBVW' -O '/content/Wav2Lip/checkpoints/wav2lip.pth'
!pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl

# !pip uninstall tensorflow tensorflow-gpu
!cd Wav2Lip && pip install -r requirements.txt

#download pretrained model for face detection
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/content/Wav2Lip/face_detection/detection/sfd/s3fd.pth"

!pip install -q youtube-dl
!pip install ffmpeg-python
!pip install librosa==0.8.1

# Text to speach
!pip install TTS

from google.colab import drive
drive.mount('/content/drive')


In [1]:
#@title Access Google sheets
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [2]:
#@title Import all modules for TTS and copy base video from google dive
from TTS.utils.manage import ModelManager
from TTS.utils.synthesizer import Synthesizer
import subprocess
import concurrent.futures
import os

!cp /content/drive/MyDrive/input_vidoe.mp4 /content
!mkdir /content/output /content/outputAudio


In [3]:
#@title All model

path = "/usr/local/lib/python3.9/dist-packages/TTS/.models.json"

model_manager = ModelManager(path)

model_path, config_path, model_item = model_manager.download_model("tts_models/en/ljspeech/tacotron2-DCA")

voc_path, voc_config_path, _ = model_manager.download_model(model_item["default_vocoder"])



def textToSpeach(text, outputFile):
  syn = Synthesizer(
      tts_checkpoint=model_path,
      tts_config_path=config_path,
      vocoder_checkpoint=voc_path,
      vocoder_config=voc_config_path
  )
  outputs = syn.tts(text)
  syn.save_wav(outputs, outputFile)


def createVideo(video, audio, output):
  os.environ["VIDEO_FILE"] = video
  os.environ["AUDIO_FILE"] = audio
  os.environ["OUTPUT_FILE"] = output
  print("calling Wav2Lip model")
  subprocess.call(['bash', '-c', 'cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face $VIDEO_FILE --audio $AUDIO_FILE --outfile $OUTPUT_FILE'])
  # !cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face video --audio audio --outfile output


def makeFileLocation(name, isAudio = False):
  return "/content/" + ("outputAudio/" if isAudio else "output/") +name


def makeVid(text, outputVideo, outputAudio = "/content/output.wav", inputVideo = "/content/input_vidoe.mp4"):

  # Text to speech
  textToSpeach(text, outputAudio)

  # create video
  createVideo(inputVideo, outputAudio, outputVideo)


def worker(rows, rowIndex):
  text = rows[rowIndex][0]
  rowIndexStr = str(rowIndex)
  
  print("Processing ", text)
  makeVid(text, makeFileLocation(rowIndexStr+".mp4"), makeFileLocation(rowIndexStr+".wav", True))


def processRows(rows, lowerLimit = 0, upperLimit = 0):
  
  if(lowerLimit == 0, upperLimit == 0):
    print("looping through the entire rows")
    upperLimit = len(rows)

  if(len(rows) < upperLimit or len(rows) < lowerLimit or lowerLimit > upperLimit): 
    print("given limit more than rows")
    return

  threads = []
  with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    for rowIndex in range(lowerLimit, upperLimit):
      threads.append(executor.submit(worker, rows, rowIndex))
  
    # Wait for the tasks to complete
    for future in concurrent.futures.as_completed(threads):
        try:
            # Get the result of the task (if any)
            result = future.result()
        except Exception as e:
            # Handle any exceptions that occurred in the task
            print(f'Task raised an exception: {e}') 


 > Downloading model to /root/.local/share/tts/tts_models--en--ljspeech--tacotron2-DCA
 > Model's license - MPL
 > Check https://www.mozilla.org/en-US/MPL/2.0/ for more info.
 > Downloading model to /root/.local/share/tts/vocoder_models--en--ljspeech--multiband-melgan
 > Model's license - MPL
 > Check https://www.mozilla.org/en-US/MPL/2.0/ for more info.


In [4]:
#@title Execute

worksheet = gc.open('AI data').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

processRows(rows)

[['Why did the tomato turn red? Because it saw the salad dressing!'], ['Why did the scarecrow win an award? Because he was outstanding in his field!'], ['Why did the bicycle fall over? Because it was two-tired!'], ["Why don't skeletons fight each other? They don't have the guts!"], ["Why don't scientists trust atoms? Because they make up everything!"], ['What do you call a fake noodle? An impasta!'], ["I'm reading a book on the history of glue. I just can't seem to put it down!"], ['Why did the coffee file a police report? It got mugged!'], ['I told my wife she was drawing her eyebrows too high. She looked surprised.'], ["What's brown and sticky? A stick!"]]
looping through the entire rows
Processing Processing  Why did the scarecrow win an award? Because he was outstanding in his field!
Processing  Why did the bicycle fall over? Because it was two-tired! 
Why did the tomato turn red? Because it saw the salad dressing!
 > Using model: Tacotron2
 > Using model: Tacotron2
 > Using model:

In [5]:
#@title zip and Downlaod output

!zip -r /content/output.zip /content/output

from google.colab import files
files.download('/content/output.zip')

  adding: content/output/ (stored 0%)
  adding: content/output/3.mp4 (deflated 0%)
  adding: content/output/4.mp4 (deflated 0%)
  adding: content/output/1.mp4 (deflated 0%)
  adding: content/output/2.mp4 (deflated 0%)
  adding: content/output/8.mp4 (deflated 0%)
  adding: content/output/9.mp4 (deflated 0%)
  adding: content/output/0.mp4 (deflated 0%)
  adding: content/output/6.mp4 (deflated 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

\\

---

# **Utils**





In [ ]:
#@title 1.Play result video -  50% scaling
inputVideoFile = "0.mp4" #@param {type:"string"}
from IPython.display import HTML
from base64 import b64encode
mp4 = open(inputVideoFile,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="50%" height="50%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

In [ ]:
#@title 2.Download Result.mp4 to your computer
fileName = "/content/0.wav" #@param {type:"string"}
from google.colab import files
files.download(fileName) 


In [13]:
#@title 3.Play audio
audioFile = "0.wav" #@param {type:"string"}
from IPython.display import Audio
Audio(audioFile, autoplay=True)

In [ ]:
#@title Clean up output
!rm -r /content/output /content/outputAudio
!mkdir /content/output /content/outputAudio